In [1]:
import os
import sqlite3
from datetime import datetime
import pandas as pd

print("Librerías importadas correctamente")

Librerías importadas correctamente


In [3]:
# Establecemos la conexión con la base de datos
conexion= sqlite3.connect('Data/Bootcamp.sql')

# Creamos un cursor que manipule la bbdd
cursor= conexion.cursor()

print("Conexión establecida exitosamente")
print(f"Base de datos: Bootcamp.sql")

Conexión establecida exitosamente
Base de datos: Bootcamp.sql


In [5]:
# Importamos los csv con los datos del ejercicio
df_alumnos_1= pd.read_csv('Data/Clase_1.csv', sep= ";")
df_alumnos_2= pd.read_csv('Data/Clase_2.csv', sep= ";")
df_alumnos_3= pd.read_csv('Data/Clase_3.csv', sep= ";")
df_alumnos_4= pd.read_csv('Data/Clase_4.csv', sep= ";")
df_profesores= pd.read_csv('Data/Claustro.csv', sep= ";")

In [6]:
df_alumnos= pd.concat([df_alumnos_1, df_alumnos_2, df_alumnos_3, df_alumnos_4], ignore_index= True)
df_alumnos.head(5)

,Nombre,Email,Promoción,Fecha_comienzo,Campus,Proyecto_HLF,Proyecto_EDA,Proyecto_BBDD,Proyecto_ML,Proyecto_Deployment,Proyecto_WebDev,Proyecto_FrontEnd,Proyecto_Backend,Proyecto_React,Proyecto_FullSatck
0,Jafet Casals,Jafet_Casals@gmail.com,Septiembre,18/09/2023,Madrid,Apto,No Apto,Apto,Apto,Apto,NaN,NaN,NaN,NaN,NaN
1,Jorge Manzanares,Jorge_Manzanares@gmail.com,Septiembre,18/09/2023,Madrid,Apto,No Apto,Apto,Apto,Apto,NaN,NaN,NaN,NaN,NaN
2,Onofre Adadia,Onofre_Adadia@gmail.com,Septiembre,18/09/2023,Madrid,Apto,Apto,Apto,No Apto,Apto,NaN,NaN,NaN,NaN,NaN
3,Merche Prada,Merche_Prada@gmail.com,Septiembre,18/09/2023,Madrid,Apto,No Apto,No Apto,Apto,No Apto,NaN,NaN,NaN,NaN,NaN
4,Pilar Abella,Pilar_Abella@gmail.com,Septiembre,18/09/2023,Madrid,Apto,No Apto,Apto,Apto,Apto,NaN,NaN,NaN,NaN,NaN
